In [1]:
%matplotlib inline

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
# import matplotlib.pyplot as plt
# import missingno

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
import catboost
import xgboost as xgb
import sklearn
# import random
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model, cross_validation
from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
# from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor, Pool, cv


# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
# 28
# BB49B3789EC2F1C13237
# 524BDEB74EB288A546

C:\Users\Dhruv\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Dhruv\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv(r'C:\Users\Dhruv\Desktop\TCD\Year 5\Machine Learning\Individual Task\test_data.csv')
test = pd.read_csv(r'C:\Users\Dhruv\Desktop\TCD\Year 5\Machine Learning\Individual Task\tcd ml 2019-20 income prediction test (without labels).csv')
submissions_file = pd.read_csv('./tcd ml 2019-20 income prediction submission file.csv')

In [3]:
train['Gender'].fillna('unknown', inplace=True)
test['Gender'].fillna('unknown', inplace=True)

In [4]:
train['Age'].fillna(train['Age'].mode()[0],inplace=True)
test['Age'].fillna(test['Age'].mode()[0], inplace=True)

In [5]:
train['Profession'].fillna(train['Profession'].mode()[0], inplace=True)
test['Profession'].fillna(test['Profession'].mode()[0], inplace=True)

In [6]:
train['University Degree'].fillna(train['University Degree'].mode()[0], inplace=True)
test['University Degree'].fillna(test['University Degree'].mode()[0], inplace=True)

In [7]:
train['Hair Color'].fillna('Unknown', inplace=True)
test['Hair Color'].fillna('Unknown', inplace=True)

In [8]:
train['flag'] = 1
test['flag'] = 0
together_df = pd.DataFrame()

In [9]:
together_df = train.append(test, ignore_index=True, sort=False)
le = preprocessing.LabelEncoder()

In [10]:
together_df['Profession'].fillna('unknown', inplace=True)

In [11]:
train

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR,flag
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.94416,1
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.32764,1
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.17670,1
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.75793,1
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.16217,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111988,111989,1980.0,female,29.0,Honduras,205274,industrial program compliance analyst,0,0,Brown,135,17185.12970,1
111989,111990,2006.0,female,36.0,Tunisia,1012826,senior policy advisor,Master,1,Brown,147,118806.53920,1
111990,111991,1993.0,female,36.0,Sri Lanka,60848,messenger,Master,1,Brown,153,289951.32940,1
111991,111992,2019.0,other,54.0,Denmark,848640,senior case support associate,No,0,Black,190,100046.52780,1


In [12]:
# train['Profession'].fillna('unknown', inplace=True);
unique_prof = together_df['Profession'].unique()

for job in unique_prof:
    row = train.loc[train['Profession'] == job]
    mean = (row.iloc[:,-2]).mean()
    
    together_df['Profession'].replace(job,mean,inplace = True)
    
avg = train['Income in EUR'].mean()
# together_df['Profession'].fillna(avg, inplace = True)

In [13]:
print(together_df['Profession'])

0         151466.154967
1         133995.974029
2         117286.626556
3          56144.047562
4          88209.535957
              ...      
185218     93805.279900
185219    110572.416009
185220     85630.251627
185221     76391.119164
185222    104782.474123
Name: Profession, Length: 185223, dtype: float64


In [14]:
unique_countries = together_df['Country'].unique()

for country in unique_countries:
    row = train.loc[train['Country'] == country]
    mean = (row.iloc[:,-2]).mean()
    
    together_df['Country'].replace(country,mean,inplace = True)

avg = train['Income in EUR'].mean()
together_df['Profession'].fillna(avg, inplace = True)

In [15]:
# print(together_df['Country'])
# together_df['Country'].replace('Iceland', 10000, inplace=True)

In [16]:
# le.fit(together_df['Profession'])
# together_df['Profession'] = le.transform(together_df['Profession'])

gender_onehot = pd.get_dummies(together_df['Gender'],prefix='gender')
# country_onehot = pd.get_dummies(together_df['Country'],prefix='country')
degree_onehot = pd.get_dummies(together_df['University Degree'],prefix='degree')
hair_onehot = pd.get_dummies(together_df['Hair Color'],prefix='hair')

In [17]:
together_df = pd.concat([together_df, gender_onehot, degree_onehot, hair_onehot], axis=1)
together_df.drop(['Gender','Hair Color','Income','University Degree'],axis=1,inplace=True)
train_df = together_df[together_df['flag'] == 1]
test_df = together_df[together_df['flag'] == 0]
train_df.set_index('Instance')
test_df.set_index('Instance')

,Year of Record,Age,Country,Size of City,Profession,Wears Glasses,Body Height [cm],Income in EUR,flag,gender_0,...,degree_Bachelor,degree_Master,degree_No,degree_PhD,hair_0,hair_Black,hair_Blond,hair_Brown,hair_Red,hair_Unknown
Instance,,,,,,,,,,,,,,,,,,,,,
111994,1992.0,21.0,40478.734970,391652,157420.026177,1,153,NaN,0,0,...,0,1,0,0,0,0,0,1,0,0
111995,1986.0,34.0,46831.726578,33653,64566.760161,0,163,NaN,0,0,...,1,0,0,0,0,1,0,0,0,0
111996,1994.0,53.0,62039.079598,34765,71989.175351,1,153,NaN,0,0,...,1,0,0,0,0,0,1,0,0,0
111997,1984.0,29.0,239513.861520,1494132,79377.161405,0,154,NaN,0,1,...,0,0,1,0,0,1,0,0,0,0
111998,2007.0,17.0,26691.192820,120661,133427.795055,0,191,NaN,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185219,2012.0,37.0,87116.830844,62975,93805.279900,0,203,NaN,0,0,...,1,0,0,0,0,0,0,1,0,0
185220,2016.0,42.0,72324.040406,46819,110572.416009,0,181,NaN,0,0,...,0,0,1,0,0,1,0,0,0,0
185221,2019.0,50.0,46257.979683,525072,85630.251627,1,162,NaN,0,0,...,0,1,0,0,0,0,0,1,0,0


In [18]:
test_df.drop(['Income in EUR'],axis=1,inplace=True)

In [19]:
temp = train_df.sort_values(by=['Income in EUR'])
temp = temp[:-8]

In [20]:
train.data = pd.DataFrame()
train_data = temp.drop('Income in EUR', axis=1)
y = temp['Income in EUR']

In [21]:
# X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_data,y, test_size=0.2)

In [22]:
# X_train = preprocessing.scale(X_train)
# test_df = preprocessing.scale(test_df)

In [23]:
# regr_rf = CatBoostRegressor(iterations=7500)
# model_rf = regr_rf.fit(X_train, y_train)
# acc = (model_rf.score(X_test,y_test))


In [24]:
# print(acc)

In [25]:
# train_df

In [26]:
# cat_features = np.where(train_data.dtypes != np.float)[0]

In [27]:
# train_pool = Pool(train_data, y, cat_features)
catboost_model = CatBoostRegressor(iterations=1750)

In [28]:
catboost_model = catboost_model.fit(train_data, y)

0:	learn: 144547.1456037	total: 200ms	remaining: 5m 49s
1:	learn: 141561.1748690	total: 309ms	remaining: 4m 29s
2:	learn: 138701.3966961	total: 416ms	remaining: 4m 2s
3:	learn: 136008.9337410	total: 533ms	remaining: 3m 52s
4:	learn: 133337.8104385	total: 614ms	remaining: 3m 34s
5:	learn: 130790.1871191	total: 690ms	remaining: 3m 20s
6:	learn: 128283.8740656	total: 762ms	remaining: 3m 9s
7:	learn: 125879.2976486	total: 849ms	remaining: 3m 4s
8:	learn: 123575.6776848	total: 941ms	remaining: 3m 1s
9:	learn: 121338.5638556	total: 1.02s	remaining: 2m 57s
10:	learn: 119131.1152769	total: 1.09s	remaining: 2m 53s
11:	learn: 117060.9063497	total: 1.16s	remaining: 2m 47s
12:	learn: 115001.2182600	total: 1.25s	remaining: 2m 47s
13:	learn: 113058.6330734	total: 1.32s	remaining: 2m 43s
14:	learn: 111182.7667836	total: 1.39s	remaining: 2m 40s
15:	learn: 109370.2250571	total: 1.45s	remaining: 2m 37s
16:	learn: 107620.4090820	total: 1.53s	remaining: 2m 36s
17:	learn: 105909.5699036	total: 1.62s	remain

146:	learn: 57352.3929447	total: 14.6s	remaining: 2m 39s
147:	learn: 57308.2967604	total: 14.8s	remaining: 2m 39s
148:	learn: 57250.2866830	total: 14.8s	remaining: 2m 39s
149:	learn: 57193.8489233	total: 14.9s	remaining: 2m 39s
150:	learn: 57139.8979165	total: 15s	remaining: 2m 38s
151:	learn: 57093.6766984	total: 15s	remaining: 2m 38s
152:	learn: 57038.1862162	total: 15.2s	remaining: 2m 38s
153:	learn: 57002.4169282	total: 15.2s	remaining: 2m 37s
154:	learn: 56954.6531927	total: 15.3s	remaining: 2m 37s
155:	learn: 56923.3808315	total: 15.4s	remaining: 2m 37s
156:	learn: 56888.6030729	total: 15.5s	remaining: 2m 37s
157:	learn: 56843.1239311	total: 15.6s	remaining: 2m 37s
158:	learn: 56796.8919324	total: 15.7s	remaining: 2m 37s
159:	learn: 56750.3761534	total: 15.8s	remaining: 2m 37s
160:	learn: 56712.2064531	total: 15.9s	remaining: 2m 37s
161:	learn: 56670.5870235	total: 16s	remaining: 2m 36s
162:	learn: 56630.4808401	total: 16.1s	remaining: 2m 36s
163:	learn: 56596.9270539	total: 16.2

293:	learn: 54126.6188593	total: 26.3s	remaining: 2m 10s
294:	learn: 54115.2701542	total: 26.3s	remaining: 2m 9s
295:	learn: 54105.3914972	total: 26.4s	remaining: 2m 9s
296:	learn: 54096.7595828	total: 26.4s	remaining: 2m 9s
297:	learn: 54089.4059216	total: 26.5s	remaining: 2m 9s
298:	learn: 54079.1098253	total: 26.6s	remaining: 2m 8s
299:	learn: 54068.0410316	total: 26.6s	remaining: 2m 8s
300:	learn: 54057.7889310	total: 26.7s	remaining: 2m 8s
301:	learn: 54043.5529572	total: 26.7s	remaining: 2m 8s
302:	learn: 54036.4527264	total: 26.8s	remaining: 2m 8s
303:	learn: 54023.9736659	total: 26.9s	remaining: 2m 7s
304:	learn: 54014.7485358	total: 26.9s	remaining: 2m 7s
305:	learn: 54006.0803864	total: 27s	remaining: 2m 7s
306:	learn: 53998.2791273	total: 27.1s	remaining: 2m 7s
307:	learn: 53988.6525303	total: 27.1s	remaining: 2m 6s
308:	learn: 53974.4790166	total: 27.2s	remaining: 2m 6s
309:	learn: 53959.5161148	total: 27.2s	remaining: 2m 6s
310:	learn: 53949.0674898	total: 27.3s	remaining:

441:	learn: 53046.3818466	total: 37.2s	remaining: 1m 50s
442:	learn: 53037.5415995	total: 37.3s	remaining: 1m 49s
443:	learn: 53032.2132908	total: 37.3s	remaining: 1m 49s
444:	learn: 53028.7072117	total: 37.4s	remaining: 1m 49s
445:	learn: 53024.3882571	total: 37.5s	remaining: 1m 49s
446:	learn: 53019.3694855	total: 37.5s	remaining: 1m 49s
447:	learn: 53013.9110769	total: 37.6s	remaining: 1m 49s
448:	learn: 53004.2527591	total: 37.6s	remaining: 1m 49s
449:	learn: 52997.8779949	total: 37.7s	remaining: 1m 48s
450:	learn: 52993.1254681	total: 37.8s	remaining: 1m 48s
451:	learn: 52988.6447522	total: 37.8s	remaining: 1m 48s
452:	learn: 52983.8535031	total: 37.9s	remaining: 1m 48s
453:	learn: 52980.5776219	total: 37.9s	remaining: 1m 48s
454:	learn: 52978.3012709	total: 38s	remaining: 1m 48s
455:	learn: 52974.3079250	total: 38.1s	remaining: 1m 48s
456:	learn: 52963.7481521	total: 38.1s	remaining: 1m 47s
457:	learn: 52959.9748420	total: 38.2s	remaining: 1m 47s
458:	learn: 52955.9015973	total: 

586:	learn: 52357.2703060	total: 47.1s	remaining: 1m 33s
587:	learn: 52354.3243302	total: 47.2s	remaining: 1m 33s
588:	learn: 52350.2857379	total: 47.2s	remaining: 1m 33s
589:	learn: 52346.8997925	total: 47.3s	remaining: 1m 33s
590:	learn: 52343.7472849	total: 47.4s	remaining: 1m 32s
591:	learn: 52340.7708710	total: 47.4s	remaining: 1m 32s
592:	learn: 52335.3506371	total: 47.5s	remaining: 1m 32s
593:	learn: 52332.5848101	total: 47.6s	remaining: 1m 32s
594:	learn: 52329.8667602	total: 47.7s	remaining: 1m 32s
595:	learn: 52327.9379431	total: 47.7s	remaining: 1m 32s
596:	learn: 52322.7155408	total: 47.8s	remaining: 1m 32s
597:	learn: 52318.4897740	total: 47.9s	remaining: 1m 32s
598:	learn: 52312.5451376	total: 48s	remaining: 1m 32s
599:	learn: 52307.5680875	total: 48s	remaining: 1m 32s
600:	learn: 52303.5355197	total: 48.1s	remaining: 1m 31s
601:	learn: 52298.5883349	total: 48.2s	remaining: 1m 31s
602:	learn: 52295.6698289	total: 48.2s	remaining: 1m 31s
603:	learn: 52289.1851292	total: 48

...

1165:	learn: 50474.3432836	total: 1m 38s	remaining: 49.3s
1166:	learn: 50471.5825135	total: 1m 38s	remaining: 49.2s
1167:	learn: 50470.6710085	total: 1m 38s	remaining: 49.1s
1168:	learn: 50469.6068011	total: 1m 38s	remaining: 49s
1169:	learn: 50459.9113820	total: 1m 38s	remaining: 49s
1170:	learn: 50456.3935308	total: 1m 38s	remaining: 48.9s
1171:	learn: 50454.4648334	total: 1m 38s	remaining: 48.8s
1172:	learn: 50453.3191401	total: 1m 39s	remaining: 48.7s
1173:	learn: 50451.2757536	total: 1m 39s	remaining: 48.7s
1174:	learn: 50449.3306539	total: 1m 39s	remaining: 48.6s
1175:	learn: 50446.3186735	total: 1m 39s	remaining: 48.5s
1176:	learn: 50440.7238408	total: 1m 39s	remaining: 48.4s
1177:	learn: 50438.0603498	total: 1m 39s	remaining: 48.3s
1178:	learn: 50435.7475143	total: 1m 39s	remaining: 48.3s
1179:	learn: 50434.5461284	total: 1m 39s	remaining: 48.2s
1180:	learn: 50431.4936176	total: 1m 39s	remaining: 48.1s
1181:	learn: 50429.4772904	total: 1m 39s	remaining: 48s
1182:	learn: 50427.5

1307:	learn: 50140.9020955	total: 1m 52s	remaining: 37.9s
1308:	learn: 50138.6798909	total: 1m 52s	remaining: 37.8s
1309:	learn: 50136.1131027	total: 1m 52s	remaining: 37.7s
1310:	learn: 50132.4582539	total: 1m 52s	remaining: 37.6s
1311:	learn: 50130.1797189	total: 1m 52s	remaining: 37.5s
1312:	learn: 50128.8877328	total: 1m 52s	remaining: 37.5s
1313:	learn: 50125.7552936	total: 1m 52s	remaining: 37.4s
1314:	learn: 50122.5151380	total: 1m 52s	remaining: 37.3s
1315:	learn: 50120.3818396	total: 1m 52s	remaining: 37.2s
1316:	learn: 50117.3830600	total: 1m 53s	remaining: 37.2s
1317:	learn: 50114.9000668	total: 1m 53s	remaining: 37.1s
1318:	learn: 50113.6527384	total: 1m 53s	remaining: 37s
1319:	learn: 50109.4675515	total: 1m 53s	remaining: 36.9s
1320:	learn: 50107.9737579	total: 1m 53s	remaining: 36.8s
1321:	learn: 50106.2036486	total: 1m 53s	remaining: 36.8s
1322:	learn: 50103.6339007	total: 1m 53s	remaining: 36.7s
1323:	learn: 50100.7642547	total: 1m 53s	remaining: 36.6s
1324:	learn: 500

1451:	learn: 49816.7667277	total: 2m 4s	remaining: 25.6s
1452:	learn: 49813.6487079	total: 2m 4s	remaining: 25.5s
1453:	learn: 49811.9452116	total: 2m 4s	remaining: 25.4s
1454:	learn: 49809.5617623	total: 2m 4s	remaining: 25.3s
1455:	learn: 49806.1533379	total: 2m 4s	remaining: 25.2s
1456:	learn: 49803.6502024	total: 2m 5s	remaining: 25.2s
1457:	learn: 49800.9893909	total: 2m 5s	remaining: 25.1s
1458:	learn: 49798.0762480	total: 2m 5s	remaining: 25s
1459:	learn: 49794.9513066	total: 2m 5s	remaining: 24.9s
1460:	learn: 49791.6655787	total: 2m 5s	remaining: 24.8s
1461:	learn: 49788.6950668	total: 2m 5s	remaining: 24.7s
1462:	learn: 49786.7281914	total: 2m 5s	remaining: 24.6s
1463:	learn: 49784.4790943	total: 2m 5s	remaining: 24.6s
1464:	learn: 49782.4593603	total: 2m 5s	remaining: 24.5s
1465:	learn: 49780.0611547	total: 2m 5s	remaining: 24.4s
1466:	learn: 49777.3401610	total: 2m 6s	remaining: 24.3s
1467:	learn: 49775.5830057	total: 2m 6s	remaining: 24.2s
1468:	learn: 49773.4412928	total:

1595:	learn: 49499.5018641	total: 2m 18s	remaining: 13.3s
1596:	learn: 49497.5034359	total: 2m 18s	remaining: 13.3s
1597:	learn: 49495.5028982	total: 2m 18s	remaining: 13.2s
1598:	learn: 49493.7095920	total: 2m 18s	remaining: 13.1s
1599:	learn: 49491.6561665	total: 2m 18s	remaining: 13s
1600:	learn: 49489.6160662	total: 2m 18s	remaining: 12.9s
1601:	learn: 49488.1084833	total: 2m 18s	remaining: 12.8s
1602:	learn: 49484.9756313	total: 2m 18s	remaining: 12.7s
1603:	learn: 49483.5972664	total: 2m 18s	remaining: 12.6s
1604:	learn: 49481.0401431	total: 2m 18s	remaining: 12.6s
1605:	learn: 49478.6346672	total: 2m 19s	remaining: 12.5s
1606:	learn: 49476.5916299	total: 2m 19s	remaining: 12.4s
1607:	learn: 49473.0726336	total: 2m 19s	remaining: 12.3s
1608:	learn: 49471.5213929	total: 2m 19s	remaining: 12.2s
1609:	learn: 49468.6625488	total: 2m 19s	remaining: 12.1s
1610:	learn: 49466.0613278	total: 2m 19s	remaining: 12s
1611:	learn: 49462.4919792	total: 2m 19s	remaining: 12s
1612:	learn: 49461.2

1737:	learn: 49205.3305819	total: 2m 31s	remaining: 1.05s
1738:	learn: 49202.0450268	total: 2m 31s	remaining: 960ms
1739:	learn: 49200.0863622	total: 2m 31s	remaining: 873ms
1740:	learn: 49197.7177364	total: 2m 31s	remaining: 786ms
1741:	learn: 49196.2641617	total: 2m 32s	remaining: 698ms
1742:	learn: 49194.2892188	total: 2m 32s	remaining: 611ms
1743:	learn: 49191.9340582	total: 2m 32s	remaining: 524ms
1744:	learn: 49189.8456445	total: 2m 32s	remaining: 437ms
1745:	learn: 49188.9313746	total: 2m 32s	remaining: 349ms
1746:	learn: 49186.2159578	total: 2m 32s	remaining: 262ms
1747:	learn: 49185.2236283	total: 2m 32s	remaining: 175ms
1748:	learn: 49183.6054239	total: 2m 32s	remaining: 87.3ms
1749:	learn: 49181.8650560	total: 2m 32s	remaining: 0us


In [29]:
predictions = catboost_model.predict(test_df)
submissions_file = pd.read_csv('./tcd ml 2019-20 income prediction submission file.csv')
Submissions = pd.DataFrame()
Submissions['Instance'] = submissions_file['Instance']
Submissions['Income'] = predictions
Submissions.to_csv('./catboost8.csv',index = False)